In [2]:
# from classes import *
# from helper_functions import *
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime, time
from datetime import date
import json
import urllib.request
import random
random.seed(2022)

#for routing
import haversine as hs
import gmaps
import googlemaps
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# for configurations
from dotenv import load_dotenv
import os, re, math
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importing the dataset and configuration parameters
data = pd.read_csv('../data/test02.csv')

# FACTORY_GEO_COORD = os.getenv("FACTORY_GEO_COORD") 
# API_KEY = os.getenv("API_KEY")

# enter your api key here
# gmaps.configure(api_key=API_KEY)

In [7]:
cols=list(data.columns)

In [8]:
cols

['id',
 'job_geo_coordinate',
 'installers_required',
 'expected_job_time',
 'est_installation_date',
 'installation_date',
 'arrival_start_time',
 'arrival_end_time',
 'INSTALLER_IDs']

In [5]:
# PreProcessing
df = data[data["job_geo_coordinate"].notnull()] # Extract rows with non-empty coordinates
df[['Latitude','Longitude']] = df["job_geo_coordinate"].str.split(',',expand=True) 
df.reset_index(inplace = True,drop=True)
cols=list(df.columns)
cols=cols[:2]+cols[-2:]+cols[2:-2]
df=df[cols]
df.rename(columns={'INSTALLER_IDs': 'installer_ids'}, inplace=True)

In [7]:
df.to_csv('data/sorted_data.csv',index=False)

In [ ]:
df_ins1=df[df["installers_required"]==1]

In [ ]:
factory_coord = list(map(float,FACTORY_GEO_COORD.split(',')))

factory = {
    'location': (factory_coord[0],factory_coord[1])
}

factory_layer = gmaps.symbol_layer(
    [factory['location']], hover_text='Factory', info_box_content='Factory', 
    fill_color='white', stroke_color='red', scale=6
)

In [ ]:
num_installers = 5

In [ ]:
jobs=[]
for i in range(len(df_ins1)):
    job = { 
        'id': str(df_ins1.loc[i,'id']),
        'location': (float(df_ins1.loc[i,'lat']), float(df_ins1.loc[i,'long']))
    }
    jobs.append(job)

In [ ]:
job_locations = [job['location'] for job in jobs]
job_labels = [job['id'] for job in jobs]

jobs_layer = gmaps.symbol_layer(
    job_locations, hover_text=job_labels, 
    fill_color='white', stroke_color='black', scale=3
)

In [ ]:
fig = gmaps.figure()
fig.add_layer(factory_layer)
fig.add_layer(jobs_layer)

# fig

In [ ]:
try:
    objective = 'distance'  # distance or duration
    # Distance Matrix API takes a max 100 elements = (origins x destinations), limit to 10 x 10
    distance_matrix = build_dist_mat(factory, jobs[0:9], objective)
    df_dist_matrix = pd.DataFrame(distance_matrix)

except:
    print('Something went wrong building distance matrix.')

df_dist_matrix

In [ ]:
addr = [item['location'] for item in [factory] + jobs[0:55]]
dist_matrix = np.array(create_distance_matrix(addr))

In [ ]:
dist_matrix

In [ ]:
for index, row in tqdm(df.iterrows()):
    try:
        # Check first_date < deadline
        if first_date > row["est_installation_date"].date():
            print(index,'Unable to process dates before ' + first_date)

        #Check if enough installers available
        if row["installers_required"] > num_installers:
            print(index,'Not enough installers available for this job')

        # Calculate which installers can reach next job location earliest
        ins_start_time_nextJob = []
        ins_travel_times = []
        for installer in installers:
            time_to_travel = get_travel_time(installer.geo_coord,row["job_geo_coordinate"])
            ins_travel_times.append(time_to_travel)
            if installer.time_spent + time_to_travel + row["expected_job_time"] > 480:                       # Can add drive back time here
                # Shift to next working day
                next_date = next_working_date(installer.end_time.date())
                installer.start_time = None
                installer.end_time = datetime.datetime.combine(next_date,datetime.time()) + datetime.timedelta(hours=8)
                installer.reset_location()
                installer.reset_time_spent()

            arrival_time = installer.end_time + datetime.timedelta(minutes=time_to_travel)                   # arrival time for next job
            ins_start_time_nextJob.append((installer.id,arrival_time.strftime('%Y-%m-%d %H:%M:%S')))
        ins_start_time_nextJob = sorted(ins_start_time_nextJob,key=lambda x: x[1])

        # Installers to send
        ins_to_send = []
        for i in range(row["installers_required"]):
            ins_to_send.append(ins_start_time_nextJob[i][0])

        # Update installers' variables
        curr_max = datetime.datetime.combine(date.today(),datetime.time()) # Start time for the installer who arrives last
        for ins_id in ins_to_send:
            installer = installers[ins_id]
            curr_travel_time = get_travel_time(installer.geo_coord,row["job_geo_coordinate"])
            prev_job_end_time = installer.end_time
            installer.start_time = prev_job_end_time + datetime.timedelta(minutes=curr_travel_time)
            curr_max = max(curr_max,installer.start_time)

        for ins_id in ins_to_send:
            installer = installers[ins_id]
            curr_travel_time = get_travel_time(installer.geo_coord,row["job_geo_coordinate"])
            prev_job_end_time = installer.end_time
            installer.avail = False
            installer.start_time = curr_max
            installer.end_time = installer.start_time + datetime.timedelta(minutes=row["expected_job_time"])
            installer.time_spent += row["expected_job_time"] + curr_travel_time
            installer.geo_coord = row["job_geo_coordinate"]

        job_start_time = curr_max
        job_end_time = curr_max + datetime.timedelta(minutes=row["expected_job_time"])
        installer_ids = ins_to_send

        df.loc[index,'installation_date'] = job_start_time.date()
        df.loc[index,'arrival_start_time'] = job_start_time.time()
        df.loc[index,'arrival_end_time'] = job_end_time.time()
        df.loc[index,'INSTALLER_IDs'] = ','.join(map(str,installer_ids))

        # # print(job_start_time,'\t',job_end_time,'\t',installer_ids)
        # if df.loc[index,'installation_date'] > row["est_installation_date"]:
        #     print(index,'Cant accomodate for', row["est_installation_date"])
    except:
        # print(index)
        continue